<a href="https://colab.research.google.com/github/shivangi0520/Sentiment-Model/blob/main/Sentiment_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
#Load the data
from google.colab import files
files.upload()

In [ ]:
#Storing the data into variables
df1 = pd.read_csv('Combined_News_DJIA.csv')
df2 = pd.read_csv('upload_DJIA_table.csv')

In [ ]:
#Show the first 3 rows of data for df1
df1.head(3)

In [ ]:
df1.shape

In [ ]:
df2.head(3)

In [ ]:
df2.shape

In [ ]:
#Merge the data set on the data field
merge = df1.merge(df2, how='inner', on='Date')

In [ ]:
merge.head(3)

In [ ]:
#combine the news headlines
headlines = []
for row in range(0,len(merge.index)):
    headlines.append(' '.join(str(x) for x in merge.iloc[row,2:27]))

In [ ]:
headlines[0]

In [ ]:
#Clean the data
clean_headlines = []
for i in range(0, len(headlines)):
  clean_headlines.append(re.sub("b[(')]+", '', headlines[i] )) #remove b'
  clean_headlines[i] = re.sub('b[(")]+', '', clean_headlines[i] ) #remove b"
  clean_headlines[i] = re.sub("\'", '', clean_headlines[i] ) #remove \'

In [ ]:
#Show the combined cleaned headlines
clean_headlines[25]

In [ ]:
#Add the clean headlines to the data set
merge['Combined_News'] = clean_headlines

#Show the new column
merge['Combined_News'][0]

In [ ]:
merge.head(3)

In [ ]:
#create a functuon to get subjectivity
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

#create a function for polarity
def getPolarity(text):
  return TextBlob(text).sentiment.polarity

In [ ]:
#create two new columns 'Subjectivity' and 'Polarity'
merge['Subjectivity'] = merge['Combined_News'].apply(getSubjectivity)
merge['Polarity'] = merge['Combined_News'].apply(getPolarity)

In [ ]:
#show the new columns in the merge data set
merge.head(3)

In [ ]:
#create a function to get the sentiment score
def getSIA(text):
  sia = SentimentIntensityAnalyzer()
  sentiment = sia.polarity_scores(text)
  return sentiment

In [ ]:
#Get the sentiment scores for each day
compound = []
neg = []
neu = []
pos = []
SIA = 0

for i in range(0, len(merge['Combined_News'])):
  SIA = getSIA(merge['Combined_News'][i])
  compound.append(SIA['compound'])
  neg.append(SIA['neg'])
  neu.append(SIA['neu'])
  pos.append(SIA['pos'])

In [ ]:
#Store the sentiment scores in the merge set data
merge['Compound'] =compound
merge['Negative'] =neg
merge['Neutral'] =neu
merge['Positive'] = pos

In [ ]:
#show the merge data
merge.head(3)

In [ ]:
#Create a list of columns to keep
keep_columns = [ 'Open',  'High', 'Low', 'Volume', 'Subjectivity', 'Polarity', 'Compound', 'Negative', 'Neutral' ,'Positive',  'Label' ]
df = merge[keep_columns]
df

In [ ]:
#Create the feature data set
X = df
X = np.array(X.drop(['Label'],1))
#Create the target data set
y = np.array(df['Label'])

In [ ]:
#Split the data into 80% training and 20% testing data sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [ ]:
#Create and train the model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [ ]:
#Show the models predictions
predictions = model.predict(x_test)
predictions

In [ ]:
y_test

In [ ]:
#Show the model metrics
print( classification_report(y_test, predictions))